In [0]:
print("Healthy")

In [0]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [0]:
import pandas as pd 
import numpy as np 
import duckdb
from re import search

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
d = datetime.today() - timedelta(days = 1)
d = d.strftime('%Y-%m-%d')

date = "'" + d + "'"
print(date)


e = datetime.today() - timedelta(days = 1)
e = e.strftime('%Y-%m-%d')
end = "'" + e + "'"
print(end, ' as end')

s = datetime.today() - timedelta(days = 8)
s = s.strftime('%Y-%m-%d')
start = "'" + s + "'"
print(start, ' as start')

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders


import mysql.connector
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

DumpDB = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


In [0]:
username = data['auto-report']['email']
passwd = data['auto-report']['passwd']
print(username)

In [0]:
## Wallets list 

wl = [
('Tradegrid', '1029004186', '["paul.adeyoyin@thetradegrid.com"]'),
('Creditchek Inc', '1030522484', '["finance@creditchek.africa"]'),
]

df_wallets = pd.DataFrame(wl, columns = ['wallet_name', 'account_nos', 'email'])

In [0]:
t = []

for i in df_wallets['account_nos']:
    t.append((i[:11]))    
    
ti = ','.join([str(x) for x in t])
ti = ti.replace('[', '')

df_wallets['account_no'] = ti.split(',')

In [0]:
#df_wallets = df_wallets[33:34]
df_wallets.head()

In [0]:
Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database'],
  auth_plugin = 'mysql_native_password',
)

nam = f'''
    select
    mc.display_name as client_name,
    msa.account_no
    from m_client mc
    join m_savings_account msa on msa.client_id = mc.id
    where msa.account_no in ({ti})
    '''

df_nam = pd.read_sql_query(nam, Mifosdb)

df_wallets = pd.merge(df_wallets, df_nam, on = 'account_no', how = 'left')

df_wallets = df_wallets[['client_name', 'wallet_name', 'account_no', 'account_nos', 'email']]

df_wallets['show_beneficiary'] = ['Yes' if i == '1011974622' else 'No' for i in df_wallets['account_no']]
df_wallets.head()


In [0]:
def statement_spool(account_no, show_beneficiary):
    
    show_beneficiary = show_beneficiary.upper()
    
    if show_beneficiary == 'NO':
        comm = '-- '
    elif show_beneficiary == 'YES':
        comm = ''
    
    ptype = pd.read_sql_query(f'''
    select 
    sa.account_no,
    sa.product_id
    from m_savings_account sa 
    where sa.account_no in ({account_no})
    ''', Mifosdb)
    
    product_id = ptype['product_id'][0]
    
    inf = f'''
    select
    st.id as transaction_id,
    mpd.receipt_number as session_id,
    st.created_date as transaction_date,
    msa.account_no as account_no,
    msp.name as product,

    ifnull(replace(mc.display_name, "null", " "), mg.display_name) as client_name,
    
    renum.enum_value as transaction_type,
    {comm}mpd.account_number as beneficiary_account_no,
    
    ifnull(case when st.transaction_type_enum in (2,4,17,18,26)  then st.amount end ,0) as 'debit', 
    ifnull(case  when st.transaction_type_enum in (1,3) then st.amount end,0) as 'credit', 
    ifnull(st.running_balance_derived,0) as 'balance', 
    case st.is_reversed when 0 then 'No' else 'Yes' end as 'reversed',
     
    case when renum.enum_value like '%Stamp Duty%' then 'Stamp Duty' 
        when st.transaction_type_enum in (20, 21) then renum.enum_message_property else mtr.remarks end as 'narration'
    
        
    
    from m_savings_account_transaction st
    left join m_savings_account msa on msa.id = st.savings_account_id
    left join m_savings_product msp on msp.id = msa.product_id
    left join (SELECT
                rv.enum_name,
                rv.enum_id,
                rv.enum_message_property,
                rv.enum_value
                FROM r_enum_value rv
                WHERE rv.enum_name = 'savings_transaction_type_enum') renum on renum.enum_id = st.transaction_type_enum
 
    
    left join m_client mc on mc.id = msa.client_id
    left join m_group mg on mg.id = msa.group_id
    left JOIN m_payment_detail mpd ON mpd.id = st.payment_detail_id
    left JOIN m_payment_type mpt ON mpt.id = mpd.payment_type_id
    left join m_transaction_request mtr on mtr.transaction_id = st.id
    where st.transaction_type_enum in ('1', '2', '3', '4', '17', '18', '26') -- ('1', '2', '3', '5', '4', '17', '18', '26')
    and left(st.created_date, 10) >= {start}  
    and msa.product_id = '{product_id}'
    and msa.account_no in ({account_no})
    -- and st.is_reversed = 0
    order by st.id
    '''

    df1 = pd.read_sql_query(inf, Mifosdb)

    import pymysql
    mifos_conn = pymysql.connect(user = data['mifos']['user'], 
                            password = data['mifos']['passwd'], 
                            database = data['mifos']['database'], 
                            host = data['mifos']['host'])
    mifos_cursor = mifos_conn.cursor()

    sql =f"""
        SELECT msat.id AS 'Tranaction ID', msa.account_no as 'Account No', msat.amount as 'Amount', 
        mtr.notes AS 'Transaction Remark', msat.created_date AS 'Transaction Date'
        FROM m_transaction_request mtr 
        JOIN m_savings_account_transaction msat ON msat.id = mtr.transaction_id
        JOIN m_savings_account msa ON msa.id = msat.savings_account_id
        WHERE msa.account_no in ({account_no})
        AND mtr.notes LIKE '%Reversal Of transactionId%'
        and left(msat.created_date, 10) >= {start}
        ORDER BY msat.id ASC;
    """
    mifos_cursor.execute(sql)
    df2 = pd.DataFrame(mifos_cursor.fetchall())

    df2 = df2.rename(columns={
    0: "Tranaction ID",
    1: "Account No",
    2: "Amount",
    3: "Transaction Remark",
    4: "Transaction Date",
    })
    
    return df1, df2


In [0]:
# fetch transaction references from wallet table and merge with output
def merge_trf(output):
    t = output[0]
    s = t['session_id'].fillna(0).astype(str)
    s = "'"+s+"'"
    trf = pd.read_sql_query(f'''
                select
                distinct session_id,
                transaction_id as Transaction_reference
                from TM_WALLET_TRANSACTIONS
                where session_id in ('0', {','.join([str(x) for x in s.tolist()])})
                group by 1
                ''', DumpDB)

    trf = pd.merge(t, trf, on ='session_id', how = 'left')
    
    new_output = [trf, output[1]]
    
    return new_output

In [0]:
def save_send_statement(merchant_name, client_name, output):
    df1 = output[0]
    df2 = output[1]
    
    with pd.ExcelWriter(f'/Workspace/ReportDump/Wallet_Account_Statements/{merchant_name}_Stmt_{start}_{end}.xlsx') as writer:
        df1.to_excel(writer,sheet_name = 'Bank Statement', index = False)
        df2.to_excel(writer,sheet_name = 'Reversal', index = False)

        
    ## FILE TO SEND AND ITS PATH
    filename = f'{merchant_name}_Stmt_{start}_{end}.xlsx'
    SourcePathName  = '/Workspace/ReportDump/Wallet_Account_Statements/' + filename 

    
    msg = MIMEMultipart()
    msg['From'] = 'auto-report@vfdtech.ng'
    msg['To'] = f'{recipient}'
    msg['CC'] = 'Olugbenga.Paseda@vfdtech.ng, Victor.Nwaka@vfd-mfb.com'
    msg['BCC'] = 'ikechukwu.mbaliri@vfd-mfb.com, data-team@vfdtech.ng'

    msg['Subject'] = 'V Account Statement'
    body = f"""
    Hello {client_name},

    Your bank statement and reversals from {start} to {end} is attached for your review.

    Regards,
    """
    msg.attach(MIMEText(body, 'plain'))

    ## ATTACHMENT PART OF THE CODE IS HERE
    attachment = open(SourcePathName, 'rb')
    part = MIMEBase('application', "octet-stream")
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(part)

    server = smtplib.SMTP('smtp.office365.com', 587)  ### put your relevant SMTP here
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(username, passwd)  ### if applicable
    server.send_message(msg)
    server.quit()
    
    return (f'{merchant_name} statement sent successfully!')


for row in df_wallets[:].itertuples():
    
    merchant_name = row[2]
    client_name = row[1]
    show_beneficiary = row[6]
        
    if search('\[', row[4]) and search(']', row[4]):
        a = row[4]
        a = a.replace('[', '')
        a = a.replace(']', '')
        
    else:
        a = row[4]

    if search('\[', row[5]) and search(']', row[5]):
        recipient = row[5]
        recipient = recipient.replace('[', '')
        recipient = recipient.replace(']', '')
        recipient = ''.join([str(x) for x in recipient.split('"')])
        
    else:
        recipient = row[5]
    
    print(show_beneficiary)     
    print(a)
    print(recipient)
    print('\n')
       
    output = statement_spool(a, show_beneficiary)
    
    if merchant_name in ['Fusion', 'Sarepay']:
        
        new_output = merge_trf(output)
    
    else:
        
        new_output = output
    
    save_send_statement(merchant_name, client_name, new_output)
    print(f'{merchant_name} statement sent successfully!')

    
